In [1]:
import json
import pandas as pd
import re
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

from keras.preprocessing import sequence
from keras.utils import np_utils

import os
import shutil
import urllib3
import zipfile
import pandas as pd
import json

In [2]:
f = open("[라벨]경상도_학습데이터_1/DKCI20000001.json", encoding="UTF-8")
raw_data = json.loads(f.read())

In [84]:
# 사투리, 표준어 사전 생성 함수
def make_dict(raw_data):

    dialect_word = []
    standard_word = [] 
    
    
    for i in range(len(raw_data["utterance"])):
        eojeol = raw_data["utterance"][i]["eojeolList"]
            
        for k in range(len(eojeol)):
            if eojeol[k]["isDialect"] == True:
                if k not in dialect_word:
                    dialect_word.append(eojeol[k]['eojeol'])
                    standard_word.append(eojeol[k]['standard'])

    word_dictionary = pd.DataFrame([dialect_word, standard_word])
    word_dictionary = word_dictionary.transpose()
    word_dictionary.columns = ['dialect', 'standard']
    dialect_dictionary = word_dictionary['dialect']
    standard_dictionary = word_dictionary['standard']
    dialect_dict = dialect_dictionary.to_dict()
    standard_dict = standard_dictionary.to_dict()

    index_to_dialect = {v:k for k,v in dialect_dict.items()}
    index_to_standard = {v:k for k,v in standard_dict.items()}

    dialect_to_index = {v:k for k,v in index_to_dialect.items()}
    standard_to_index = {v:k for k,v in index_to_standard.items()}

    print(len(index_to_dialect))
    print(len(dialect_to_index))

    print(len(index_to_standard))
    print(len(standard_to_index))
    

    return dialect_to_index,standard_to_index,index_to_dialect,index_to_standard
   

In [ ]:
index_to_dialect,index_to_standard,dialect_to_index,standard_to_index = make_dict(raw_data)

In [13]:
# 사투리 표준어 쌍 생성 함수
def make_pair(raw_data):
    filter = re.compile('[^가-힣+]')

    dialect_form = []
    standard_form = []

    for i in range(len(raw_data["utterance"])):
        sentence = raw_data["utterance"][i]
        
        for j in range(len(sentence["eojeolList"])):
            #print(sentence["eojeolList"][0]['eojeol'])
            if sentence["eojeolList"][j]['isDialect'] == True:
                try : 

                    dialect = sentence["eojeolList"][j]["eojeol"]
                    standard = sentence["eojeolList"][j]["standard"]
                    
                    dialect = filter.sub('', dialect)
                    standard = filter.sub('', standard)
                    
                    dialect = dialect_to_index[dialect]
                    standard = standard_to_index[standard]

                    dialect_form.append(dialect)
                    standard_form.append(standard)
                except:
                    pass

    result = pd.DataFrame(data = {"dialect_words":dialect_form, "standard_words": standard_form})


    return result
               

In [ ]:
pairs = make_pair(raw_data)

In [ ]:
# 사투리 사전에 존재하는지 확인하는 함수

def exist_in_dialect(x):
    for i in range(len(x)):
        for j in range(len(dialect_to_index)):
            if i==j:
                return True
            else:
                return False



In [19]:
xtrain = np_utils.to_categorical(pairs["dialect_words"])
ytrain = np_utils.to_categorical(pairs["standard_words"])
len(xtrain[0])

416

In [151]:
xtrain.shape

(398, 416)

In [158]:
model = Sequential()
model.add(Dense(256, input_dim = 416, activation="relu"))
model.add(Dense(416, activation="softmax"))

In [159]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [160]:
model.fit(xtrain, ytrain, batch_size=100, epochs=100)

Epoch 1/100
4/4 [==============================] - 0s 5ms/step - loss: 6.0152 - accuracy: 0.0101
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 5.9608 - accuracy: 0.2915
Epoch 3/100
4/4 [==============================] - 0s 5ms/step - loss: 5.9069 - accuracy: 0.4397
Epoch 4/100
4/4 [==============================] - 0s 3ms/step - loss: 5.8497 - accuracy: 0.5503
Epoch 5/100
4/4 [==============================] - 0s 3ms/step - loss: 5.7876 - accuracy: 0.5678
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 5.7184 - accuracy: 0.5879
Epoch 7/100
4/4 [==============================] - 0s 5ms/step - loss: 5.6426 - accuracy: 0.5930
Epoch 8/100
4/4 [==============================] - 0s 4ms/step - loss: 5.5567 - accuracy: 0.5854
Epoch 9/100
4/4 [==============================] - 0s 5ms/step - loss: 5.4613 - accuracy: 0.5553
Epoch 10/100
4/4 [==============================] - 0s 3ms/step - loss: 5.3540 - accuracy: 0.5327
Epoch 11/100
4/4 [===========

In [192]:
# 문장 번역 함수
def translate(sentence):
    res = ""

    sentence = sentence.split()

    for i in range(len(sentence)):
        if exist_in_dialect(sentence[i]) == True:
            index = dialect_to_index[sentence[i]]
            one_hot = np_utils.to_categorical(index,len(xtrain[0])) #인덱스 배열의크기 지정 필요 size_of_dialect
            pred_index = np.argmax(model.predict(np.array([one_hot])))
            res = res + " " + index_to_standard[pred_index]

        else:
            res = res + " " + sentence[i]

    return res
